# Trending Movie Data Analysis and Database Design 

## part 1 Clean Source Data

We create a movie database with a dataset about 1000 highly rated movies and scraped data from Youtube & Twitter.

In [4]:
from __future__ import division
from datetime import datetime
import requests
from lxml import html, etree
import json
import pandas as pd
import matplotlib.pyplot as plt
import warnings

## Data Cleaning

Initially, we have a CSV table contains all the information about a movie, from actor to income and investment. To conform with our database ER diagram, we seperate them into 4 tables and use the movie name as Primary key and foreign key.

In [5]:
# Import 4 csv tables 
movieMain=pd.read_csv("datasets/movie_main.csv",encoding='latin1')
# Movie_data stands for the major feature about the movie, such as the movie title,duration,genre.
movieActor=pd.read_csv("datasets/movie_actor.csv",encoding='latin1')
movieDirector=pd.read_csv("datasets/movie_director.csv",encoding='latin1')
moviePerformance=pd.read_csv("datasets/movie_performance.csv",encoding='latin1')

In [6]:
# Show movie genre in movie_main.csv
movie_genre=movieMain['Genre']
print(movie_genre[0:10])

0       Action,Adventure,Sci-Fi
1      Adventure,Mystery,Sci-Fi
2               Horror,Thriller
3       Animation,Comedy,Family
4      Action,Adventure,Fantasy
5      Action,Adventure,Fantasy
6            Comedy,Drama,Music
7                        Comedy
8    Action,Adventure,Biography
9       Adventure,Drama,Romance
Name: Genre, dtype: object


In [7]:
# Use the first keyword as the main genre of the movie
for r in range(1000):
    location=movie_genre[r].find(',')
    if(location!=-1):
        movie_genre[r]=movie_genre[r][0:location]

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
print (movie_genre[0:10])

0       Action
1    Adventure
2       Horror
3    Animation
4       Action
5       Action
6       Comedy
7       Comedy
8       Action
9    Adventure
Name: Genre, dtype: object


In [9]:
# Seperate actors from actor column
actor_row=movieActor['Actors']

movieActor['Actor_2']=movieActor['Actors'].str.split(',').str.get(1)
movieActor['Actor_3']=movieActor['Actors'].str.split(',').str.get(2)
movieActor['Actors']=movieActor['Actors'].str.split(',').str.get(0)
print(movieActor[0:1000:99])

     Rank                Actors           Actor_2             Actor_3
0       1           Chris Pratt        Vin Diesel      Bradley Cooper
99    100     Leonardo DiCaprio        Matt Damon      Jack Nicholson
198   199  Benedict Cumberbatch   Keira Knightley       Matthew Goode
297   298         Anne Hathaway      Meryl Streep      Adrian Grenier
396   397         Edward Norton         Liv Tyler            Tim Roth
495   496            Will Smith       Alice Braga       Charlie Tahan
594   595          Will Ferrell     Mark Wahlberg    Linda Cardellini
693   694         Kit Harington    Emily Browning   Kiefer Sutherland
792   793            Seth Rogen   Elizabeth Banks      Craig Robinson
891   892             Paul Rudd       Leslie Mann        Maude Apatow
990   991           Rhona Mitra     Michael Sheen          Bill Nighy


In [10]:
# Show the validity of the data. Rating scores range from 1.9 to 9.0. There are no invalid rating data in the datasete.
moviePerformance['Rating'].value_counts().sort_index()

1.9     1
2.7     2
3.2     1
3.5     2
3.7     2
3.9     3
4.0     1
4.1     1
4.2     2
4.3     4
4.4     1
4.5     1
4.6     5
4.7     6
4.8     4
4.9     7
5.0     4
5.1     5
5.2    11
5.3    12
5.4    12
5.5    14
5.6    17
5.7    21
5.8    26
5.9    19
6.0    26
6.1    31
6.2    37
6.3    44
6.4    35
6.5    40
6.6    42
6.7    48
6.8    37
6.9    31
7.0    46
7.1    52
7.2    42
7.3    42
7.4    33
7.5    35
7.6    27
7.7    27
7.8    40
7.9    23
8.0    19
8.1    26
8.2    10
8.3     7
8.4     4
8.5     6
8.6     3
8.8     2
9.0     1
Name: Rating, dtype: int64

In [11]:
# Show the validity of the data. Metascores range from 11 to 90. There are no invalid metascores in the datasete.
moviePerformance['Metascore'].value_counts().sort_index()

11.0      1
15.0      1
16.0      1
18.0      4
19.0      1
20.0      1
22.0      3
23.0      6
24.0      2
25.0      2
26.0      2
27.0      6
28.0      3
29.0      4
30.0      7
31.0     10
32.0      7
33.0     11
34.0     14
35.0     11
36.0     11
37.0      9
38.0      8
39.0     16
40.0     13
41.0     11
42.0     17
43.0     10
44.0     14
45.0     11
         ..
70.0     10
71.0     15
72.0     25
73.0     14
74.0     17
75.0      8
76.0     21
77.0     13
78.0     13
79.0     16
80.0     10
81.0     21
82.0     12
83.0     13
84.0      9
85.0      9
86.0      7
87.0      4
88.0      8
89.0      3
90.0      5
91.0      1
92.0      2
93.0      3
94.0      3
95.0      3
96.0      4
98.0      1
99.0      1
100.0     1
Name: Metascore, Length: 84, dtype: int64

In [14]:
# There is no null value in director dataset
movieDirector.isnull().sum()

Rank        0
Director    0
dtype: int64

In [15]:
# There is no duplicated data in director dataset
movieDirector.duplicated().sum()

0

In [16]:
# Check null value in movie performance dataset
moviePerformance.isnull().sum()

Rank                    0
Rating                  0
Votes                   0
Revenue (Millions)    128
Metascore              64
dtype: int64

In [18]:
# Fill null values with 'null'
moviePerformance.fillna('null')

,Rank,Rating,Votes,Revenue (Millions),Metascore
0,1,8.1,757074,333.13,76
1,2,7.0,485820,126.46,65
2,3,7.3,157606,138.12,62
3,4,7.2,60545,270.32,59
4,5,6.2,393727,325.02,40
5,6,6.1,56036,45.13,42
6,7,8.3,258682,151.06,93
7,8,6.4,2490,null,71
8,9,7.1,7188,8.01,78
9,10,7.0,192177,100.01,41


In [20]:
# There is no duplicated data in movie performance
moviePerformance.duplicated().sum()

0

And then we output those data into 4 csvs using pandas.to_csv().
For now, we have 4 cleaned csv tables and we can then use them as keywords to scrape data from social Media. 